In [2]:
import requests
import random
import pandas as pd 
from tqdm import tqdm
from bs4  import BeautifulSoup
import json 
import time
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [4]:
YOUR_API_KEY = os.getenv('API_KEY')

In [5]:
df = pd.read_csv('precio_mes_combustible.csv')

In [6]:
df.head(2)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado
0,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,GASOLINA CORRIENTE OXIGENADA,9860,A
1,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,BIODIESEL EXTRA,9780,A


<h1><center>Geocoding Api's</center></h1>

## Google API

* [**Google_Free_Tier**](https://cloud.google.com/free/?utm_source=google&utm_medium=cpc&utm_campaign=latam-LATAM-all-es-dr-AKWS-all-all-trial-p-dr-1011454-LUAC0013610&utm_content=text-ad-none-any-DEV_c-CRE_540885993580-ADGP_Hybrid%20%7C%20BKWS%20-%20PHR%20%7C%20Txt%20~%20Management-Tools_Console-KWID_43700065922071537-kwd-651816590614&utm_term=KW_google%20console-ST_Google%20Console&gclid=Cj0KCQjwnbmaBhD-ARIsAGTPcfWVMlJlTFywFNaAo8P-MNL4_gM6L7yucqAQ37xg1AIEHzpAzF_kkAoaAsszEALw_wcB&gclsrc=aw.ds)
* [**Google_Api_Pricing**](https://mapsplatform.google.com/pricing/)

## HERE Platform

* [**Here_Register**](https://platform.here.com/sign-up?step=verify-identity&aid=www.here.com-start-building)
* [**Here_Princing**](https://www.here.com/pricing)
* [**Geocoding_Documentation**](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics/endpoint-geocode-brief.html)

# Clean Data 

In [7]:
df[df['Periodo']==2022]['Mes'].max()

2

In [8]:
df2 =  df[(df['Periodo']==2022) & (df['Mes']==2)]
df2 =  df2[(df2['Departamento']=='ANTIOQUIA') | (df2['Departamento']=='BOGOTA D.C.')]

In [9]:
df2['Municipio'].value_counts().head(10)

BOGOTA  D.C.    1368
Name: Municipio, dtype: int64

In [10]:
df3 = df2[(df2['Departamento']=='BOGOTA D.C.') | ((df2['Departamento']=='ANTIOQUIA') & ((df2['Municipio']=='MEDELLIN') | (df2['Municipio']=='ITAGUI') | (df2['Municipio']=='BELLO') | (df2['Municipio']=='SABANETA') | (df2['Municipio']=='RIONEGRO') | (df2['Municipio']=='ENVIGADO')))]
df3.reset_index(inplace = True)
df3.drop(columns = 'index',inplace = True)

In [11]:
df3.head(2)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado
0,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,GASOLINA CORRIENTE OXIGENADA,9860,A
1,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,BIODIESEL EXTRA,9780,A


In [12]:
df3['Municipio'].unique()

array(['BOGOTA  D.C.'], dtype=object)

In [13]:
df3['Dir2'] = df3['Direccion'].apply(lambda x : x.replace('#','No'))
df3['Full_Address'] = df3.apply(lambda x : x['Direccion'] + ', ' + x['Municipio'].capitalize() + ', Colombia' , axis = 1)

In [14]:
df3['Full_Address'].nunique()

376

In [15]:
df3['Full_Address'][0]

'AVENIDA CARRERA 7 No. 123A-48, Bogota  d.c., Colombia'

In [16]:
estaciones =  df3[['Nombre_comercial','Direccion','Full_Address']].drop_duplicates(subset = 'Nombre_comercial')
estaciones.reset_index(inplace = True)
estaciones.drop(columns = 'index',inplace = True)

In [17]:
estaciones['Full_Address'][0]

'AVENIDA CARRERA 7 No. 123A-48, Bogota  d.c., Colombia'

In [18]:
address = estaciones['Full_Address'][0]
url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={YOUR_API_KEY}'
response = requests.get(url).json()

In [20]:
def Get_Coords(address,YOUR_API_KEY):
    
    url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={YOUR_API_KEY}'
    
    try : 
        
        response = requests.get(url).json()
        CleanAddress = response['items'][0]['title'].upper()
        LAT = response['items'][0]['position']['lat']
        LNG = response['items'][0]['position']['lng']
        
        results = [CleanAddress,LAT,LNG]
        
    except :
        
        results = ['Not Found','NA','NA']
        
    
    return results 

In [21]:
Get_Coords(address,YOUR_API_KEY)

['AVENIDA CARRERA 7 123A-48, 110111 BOGOTÁ, D.C., COLOMBIA',
 4.69921,
 -74.02878]

# Extracción de coordenadas 

In [22]:
coordenadas = []
tiempo = [1,2,2.5]

for i in tqdm(range(len(estaciones['Full_Address'])),colour = 'green'):
    
    try :
        
        R = Get_Coords(estaciones['Full_Address'][i],YOUR_API_KEY)
        geo_source = R[1],R[2]
        coordenadas.append(geo_source)
        
    except :
        
        coordenadas.append('Error')
        
    
    time.sleep(random.choices(tiempo)[0])
    
        

100%|██████████| 376/376 [02:18<00:00,  2.72it/s]


In [25]:
coordenadas

[(4.69921, -74.02878),
 (4.60425, -74.09239),
 ('NA', 'NA'),
 (4.67007, -74.14334),
 ('NA', 'NA'),
 (4.6015, -74.12433),
 (4.6043, -74.06738),
 (4.66158, -74.07353),
 (4.66807, -74.06893),
 (4.59476, -74.14668),
 (4.6571, -74.1379),
 (4.65303, -74.11627),
 (4.59931, -74.10626),
 (4.59388, -74.1307),
 (4.59644, -74.17597),
 (4.68731, -74.04449),
 (4.65684, -74.04844),
 (4.60204, -74.09358),
 (4.60338, -74.12261),
 (4.59297, -74.09093),
 (4.74166, -74.11302),
 (4.74739, -74.06543),
 (4.7638, -74.03136),
 (4.61608, -74.16216),
 (4.69253, -74.09836),
 (4.6114, -74.10283),
 (4.60946, -74.18085),
 (4.62009, -74.1608),
 ('NA', 'NA'),
 (4.59749, -74.1262),
 (4.65058, -74.07265),
 (4.72277, -74.03923),
 (4.68385, -74.14214),
 (4.62892, -74.08253),
 (4.67459, -74.12355),
 (4.75934, -74.04472),
 (4.61391, -74.13604),
 (4.6678, -74.14191),
 ('NA', 'NA'),
 (4.68307, -74.09824),
 ('NA', 'NA'),
 (4.67713, -74.08593),
 (4.5533, -74.09912),
 (4.62334, -74.16768),
 (4.66302, -74.0604),
 (4.78087, -74.08

In [36]:
estaciones['Coords'] = coordenadas

In [37]:
len(df3)

1368

In [38]:
estaciones.head(2)

,Nombre_comercial,Direccion,Full_Address,Coords
0,SUPER ESTACION DE SERVICIO TEXACO 10,AVENIDA CARRERA 7 No. 123A-48,"AVENIDA CARRERA 7 No. 123A-48, Bogota d.c., C...","(4.69921, -74.02878)"
1,ESTACION DE SERVICIO ROXI,Carrera 24 No 7-11,"Carrera 24 No 7-11, Bogota d.c., Colombia","(4.60425, -74.09239)"


In [32]:
df_final = pd.merge(df3,estaciones[['Full_Address','Coords']],how = 'inner' , on = 'Full_Address')

In [33]:
df_final.head(3)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado,Dir2,Full_Address,Coords
0,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,GASOLINA CORRIENTE OXIGENADA,9860,A,AVENIDA CARRERA 7 No. 123A-48,"AVENIDA CARRERA 7 No. 123A-48, Bogota d.c., C...","(4.69921, -74.02878)"
1,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,BIODIESEL EXTRA,9780,A,AVENIDA CARRERA 7 No. 123A-48,"AVENIDA CARRERA 7 No. 123A-48, Bogota d.c., C...","(4.69921, -74.02878)"
2,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,GASOLINA EXTRA OXIGENADA,17250,A,AVENIDA CARRERA 7 No. 123A-48,"AVENIDA CARRERA 7 No. 123A-48, Bogota d.c., C...","(4.69921, -74.02878)"


In [34]:
df_final = pd.merge(df3,estaciones[['Full_Address','Coords']],how = 'inner' , on = 'Full_Address')

In [35]:
df_final.to_csv('DATASET_FINAL.CSV',index = False)